As a use case, below you find a simple example of keras fitting from the datastructure just obtained. It is only for demonstration purpose, so no tuning or optimization has been done, do it yourself and please submit a PR 😉.

Workspace setup

In [ ]:
import os
from pathlib import Path
root = Path('./../')

import logging
logging.basicConfig()
log = logging.getLogger()
log.setLevel(logging.INFO)
pil_logger = logging.getLogger('PIL') # shutup PIL!
pil_logger.setLevel(logging.INFO)


# Set an example dir for images files
MOCKS = root.joinpath('./../cytokinin/cytokinin/tests/mocks/')
IMGS = MOCKS/'imgs' # this is another Path object
os.listdir(str(IMGS))

In [ ]:
import cytokinin as ck
from cytokinin.data import take_data

In [ ]:
# Create a Data from two roots: dogs and stones
dogs = take_data('images').store_filesnames_from_folder(IMGS.joinpath('dog'))
stones = take_data('images').store_filesnames_from_folder(IMGS.joinpath('stone'))
dands = dogs.copy().add_from_data(stones)

csv_url = MOCKS/'labels'/'dogsandstones_labes.csv'
dands.label_from_csv(csv_url, col='Y')
print(dands)

In [ ]:
idg_args = {
    'rescale': 1./255,
    'shear_range': 0.2,
    'zoom_range': 0.2,
    'horizontal_flip': True
}
ke_gen = dands.export_to_keras(imagedatagenerator_args=idg_args)

In [ ]:
### This part is for preparing a Keras model ####

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD, Adam

n_classes = 2

### Model definition ###
model = Sequential()
model.add( Dense(100, activation='relu', input_shape=(256,256,3)) )
model.add( Dense(60, activation='relu') )
model.add( Dense(10, activation='relu') )
model.add( Flatten() )
model.add( Dense(n_classes, activation='softmax') )

### Compile ###
sgd = SGD(lr=1e-4, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#imsize = (64,64)


... and fit!

In [ ]:
### Fit ###
model.fit_generator(
        ke_gen,
        #steps_per_epoch=20,
        epochs=3,
        #validation_data=validation_generator,
        validation_steps=1)